# In this file, a facebook network is created, agent and model classes are defined, and a sensitivity analysis is performed. The resulting data is then saved to separate files, which can be downloaded into another notebook designed for plotting.


##### Designed and created by Enno Kuyt, Susy Maijer, Sven Poelmann, Marija Puljic & Jingxin Xu.

### Importing all required packages

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
import axelrod as axl
import pandas as pd
import numpy as np
import random
import time

from mesa import Agent
from mesa import Model
from mesa.datacollection import DataCollector
from mesa.batchrunner import BatchRunner, FixedBatchRunner
from mesa.time import StagedActivation
from mesa.space import NetworkGrid

from SALib.sample import saltelli
from SALib.analyze import sobol

from itertools import combinations

from IPython.display import clear_output

### Creating the network

In [ ]:
# Import Facebook data
G1 = nx.read_edgelist("facebook_combined.txt", create_using = nx.Graph(), nodetype=int)

# key function
def deg(item) :
     return item[1]

# Arrange network list
lista=G1.degree()
lista= list(G1.degree())
lista.sort(reverse=True, key=deg)

# Remove all with more than 30 connections
remove = [i[0] for i in lista if i[1]>=35]
G1.remove_nodes_from(remove)

# Create subgraph where all have at least 4 connections
G2= nx.k_core(G1, 3)
graphs = sorted(nx.connected_components(G2), key=len, reverse=True)

# Set required variables for agent/model classes
final_network = G2.subgraph(graphs[0]).copy()
S6 = [axl.strategies[25],axl.strategies[35],axl.strategies[186],axl.strategies[95],axl.strategies[131]]
AMOUNT_TURNS = 10

### Agent class

In [ ]:
class Player(Agent):
    """
    A Player (agent) which plays PD with its neighbors and possibly adjusts its strategy. Whether he adjusts his
    strategy or not depends on the outcomes of the games, his level of rationality and the social hierarchy.
    """
    def __init__(self, unique_id, model, strategy, rationality, rank):
        super().__init__(unique_id, model)

        self.strategy = strategy()                # Set own strategy
        self.initialize_strategies_conception()   # Set all strategies + scores in self.strategies_conception
        
        # Rank
        if rank <= 0.5:
            # min of 0.0001 to prevent division by 0
            self.rank = min(max(np.random.normal(rank, (1-rank)/2), 0.0001), 1)
        elif rank > 0.5:
            # min of 0.0001 to prevent division by 0
            self.rank = min(max(np.random.normal(rank, rank/2), 0.0001), 1)
        
        # Rationality
        if rationality <= 0.5:
            self.rationality = min(max(np.random.normal(rationality, (1-rationality)/2), 0), 1)
        elif rationality > 0.5:
            self.rationality = min(max(np.random.normal(rationality, rationality/2), 0), 1)
    
    def initialize_strategies_conception(self):
        """
        initialize the conception of all the strategies for this agent
        """
        # Get the used strategies in the model 
        strategies = self.model.strategies.copy()

        # Create a dict with strategy names and their score 
        strategies_dict = {} 
        for strategy in strategies:
            strategies_dict[strategy.name] = 0 # Initial score = 0
    
        # Set the dict
        self.strategies_conception = strategies_dict
 
    def play(self):     
        """
        select a single other player in your network and play a game with them.
        """
  
        # Play a game
        random_agent =  self.model.pairings[self]
        match = axl.Match((self.strategy, random_agent.strategy), turns=AMOUNT_TURNS)
        results = match.play()
        final_score = match.final_score()
        winner = match.winner()
        
        # Count cooperations and defections
        for i in range(0, AMOUNT_TURNS):
            if results[i][0] == axl.Action.C:
                self.model.cooperators += 1
            else:
                self.model.defectors += 1 

            if results[i][1] == axl.Action.C:
                self.model.cooperators += 1 
            else:
                self.model.defectors += 1
            
        # Save the score of both
        self.score = final_score[0] / AMOUNT_TURNS
        random_agent.score = final_score[1] / AMOUNT_TURNS
        
    def evaluate(self):
        """
        evaluate the scores of this round for itself and neighboring nodes.
        the agent takes into account the score and the subjective level of respect, based on rank and rationality
        switch strategies to the best perceived strategy. 
        """
        # Update their conception of their own strategy (with the full score because he respects himself 100%)
        self.strategies_conception[self.strategy.name] += self.score
        
        # Get the neighbors
        neighbors_nodes = self.model.grid.get_neighbors(self.pos, include_center=False)
        neighbors = [agent for agent in self.model.grid.get_cell_list_contents(neighbors_nodes)]
        
        # Find normalization constants
        normalize_strategies = {} 
        for strategy in self.model.strategies.copy():
            normalize_strategies[strategy.name] = 0
        for neighbor in neighbors:  
            normalize_strategies[neighbor.strategy.name] += 1
        
        # Update the conception of the strategies of its neigbhors
        for neighbor in neighbors:
            
            # Get the social influence of the neighbor on this agent and update the view of the strategies
            friend_influence_strat = friend_influence(self, neighbor) # influence
            normalize_strat = normalize_strategies[neighbor.strategy.name] # normalizing 
            self.strategies_conception[neighbor.strategy.name] += friend_influence_strat / normalize_strat
            
        # Find best strategy in agents opinion
        self.new_strat_name = keywithmaxval(self.strategies_conception)
        
        # Add random switching to avoid quick convergence to stable point
        if random.random() < self.model.switch_random:
            self.new_strat_name = random.choice(list(self.strategies_conception.keys())) # random strategy

    def update_reset(self):
        """
        agent updates his strategy
        """
        # Set new strategy (is possible equal to previous strategy)
        self.strategy = self.model.strategies_name_mapping[self.new_strat_name]
        
        # Reset score
        self.score = 0

# Two helper functions for Agent class
def friend_influence(i, j):
    """ 
    The  influence agent j has on agent i is determined by the rationality of i and the rank/score of j.
    A rational agent focuses more on the score, while a irrational agent also looks at the rank.
    """
    return i.rationality * j.score + (1 - i.rationality) * j.score * (j.rank / i.rank)

def keywithmaxval(d):
    """ a) create a list of the dict's keys and values; 
        b) return the key with the max value
        Extended by finding all occurences and
        taking a random one of the winners"""  
    v=list(d.values())
    k=list(d.keys())
    indices = [i for i, x in enumerate(v) if x == max(v)]
    return k[np.random.choice(indices)]

### Model Class

In [ ]:
class MesaAxelrodNetwork(Model):
    def __init__(self, network=final_network, strategies=S6,
                 rationality=0.5, rank=0.5, switch_random=0.05):
        super().__init__()

        # Initialize the strategies and also create a dict to map the name to the strategy
        self.strategies = strategies  
        self.strategies_name_mapping = {
            strategy.name : strategy()
            for strategy in 
            strategies
        }
        
        # Initialize network and counters
        self.G = network # set network
        self.cooperators = 0
        self.defectors = 0
        self.switch_random = switch_random
        self.stratC = 0
        self.stratD = 0
        self.stratTFT = 0
        self.stratRandom = 0
        self.stratWSLS = 0
        self.stratGrudger = 0

        # Add a grid and scheduler
        self.grid = NetworkGrid(self.G)
        self.schedule = StagedActivation(self, ["play", "evaluate", "update_reset"])
        
        # Add datacollector
        self.datacollector = DataCollector(
            model_reporters={"Strat_Cooperate": lambda m: m.collect_strat_amount('Cooperator'),
                             "Strat_Defect": lambda m: m.collect_strat_amount('Defector'),
                             "Strat_TFT": lambda m: m.collect_strat_amount('Tit For Tat'),
                             "Strat_WSLS": lambda m: m.collect_strat_amount('Win-Stay Lose-Shift'),
                             "Strat_Random": lambda m: m.collect_strat_amount('Random'),
                             "Cooperators": lambda m: m.get_cooperators(),
                             "Defectors": lambda m: m.get_defectors()})

        # Create agents
        for i, node in enumerate(self.G.nodes()):

            # Make player with a random strategy and add to scheduler
            a = Player(i+1, self, random.choice(strategies), rationality, rank)
            self.schedule.add(a)
            
            # Add agent
            self.grid.place_agent(a, node)
        
        # Update and collect counts
        self.update_strat_counts()
        self.running = True
        self.datacollector.collect(self)

    def step(self):
        """
        Method that runs the model for a specific amount of steps.
        """ 
        # Make pairings 
        self.pairings = make_pairings(self.schedule.agents.copy())

        # Let scheduler do a step and collect
        self.reset_strat_counts()
        self.schedule.step()
        self.update_strat_counts()
        self.datacollector.collect(self)
            
    def reset_strat_counts(self):
        """
        Method that resets the counts of each strategy
        """ 
        self.stratC = 0
        self.stratD = 0
        self.stratTFT = 0
        self.stratRandom = 0
        self.stratWSLS = 0
        self.stratGrudger = 0
        self.cooperators = 0
        self.defectors = 0
    
    def update_strat_counts(self):
        """
        Method that counts the amount of occurrences of each strategy
        """ 
        for agent in self.schedule.agents:
            if agent.strategy.name == 'Cooperator':
                self.stratC += 1
            if agent.strategy.name == 'Defector':
                self.stratD += 1
            if agent.strategy.name == 'Tit For Tat':
                self.stratTFT += 1
            if agent.strategy.name == 'Random':
                self.stratRandom += 1
            if agent.strategy.name == 'Win-Stay Lose-Shift':
                self.stratWSLS += 1
            if agent.strategy.name == 'Grudger':
                self.stratGrudger += 1

    def collect_strat_amount(self, strat):
        """
        Method that returns the amount of occurrences of given strategy
        """ 
        if strat == 'Cooperator':
            return self.stratC
        if strat == 'Defector':
            return self.stratD
        if strat == 'Tit For Tat':
            return self.stratTFT
        if strat == 'Random':
            return self.stratRandom
        if strat == 'Win-Stay Lose-Shift':
            return self.stratWSLS
        if strat == 'Grudger':
            return self.stratGrudger
    
    def get_cooperators(self):
        """
        Method that returns the total amount of cooperations at that time step
        """ 
        return self.cooperators
    
    def get_defectors(self):
        """
        Method that returns the total amount of defections at that time step
        """ 
        return self.defectors
    
# Helper function for model class
def make_pairings(listForPairings):
    """
    make random pairings so that each agent plays exactly 1 other agent
    NB we implemented our own function to make semi-random lists because other functionality was really slow
    this takes += 0.001sec for a list of 1055 nodes
    """
    pair_left = listForPairings.copy()
    pair_right = listForPairings.copy()
    paired = False
    while paired == False:
        
        # Initialize
        pairings = {}
        paired = True
        np.random.shuffle(pair_left)
        np.random.shuffle(pair_right)
        
        # Do the pairings
        for i in np.arange(0, len(listForPairings)):
            if pair_left[i] != pair_right[i]:
                pairings[pair_left[i]] = pair_right[i]
            elif i < len(listForPairings)-1:
                tmp = pair_right[i+1]
                pair_right[i+1] = pair_right[i]
                pairings[pair_left[i]] = tmp
            else:
                paired = False # try again. this happens only 1 every 1000 runs and takes only 1 ms
                break

    return pairings

### Sensitivity analysis

In [ ]:
# We define our variables and bounds
problem = {
    'num_vars': 3,
    'names': ['rationality', 'rank', 'switch_random'],
    'bounds': [[0, 1], [0, 1], [0, 0.5]]
}

# Set the repetitions, the amount of steps, and the amount of distinct values per variable
replicates = 50
max_steps = 50
distinct_samples = 10

# Set the outputs
model_reporters={"Strat_Cooperate": lambda m: m.collect_strat_amount('Cooperator'),
                 "Strat_Defect": lambda m: m.collect_strat_amount('Defector'),
                 "Strat_TFT": lambda m: m.collect_strat_amount('Tit For Tat'),
                 "Strat_Random": lambda m: m.collect_strat_amount('Random'),
                 "Strat_WSLS": lambda m: m.collect_strat_amount('Win-Stay Lose-Shift'),
                 "Cooperators": lambda m: m.get_cooperators(),
                 "Defectors": lambda m: m.get_defectors()}

### OFAT

In [ ]:
# To store the data
data = {}

# For every input parameter
for i, var in enumerate(problem['names']):
    
    # Create samples
    samples = np.linspace(*problem['bounds'][i], num=distinct_samples)

    # Run Batch
    batch = BatchRunner(MesaAxelrodNetwork, 
                        max_steps=max_steps,
                        iterations=replicates,
                        variable_parameters={var: samples},
                        model_reporters=model_reporters,
                        display_progress=True)
    batch.run_all()
    
    # Store data
    data[var] = batch.get_model_vars_dataframe()

# Save data to separate files
for key in problem['names']:
    data[key].to_csv("ofat_{}.csv".format(key))

### SOBOL

In [ ]:
# Create samples
param_values = saltelli.sample(problem, distinct_samples)

# Set the outputs
model_reporters={"Strat_Cooperate": lambda m: m.collect_strat_amount('Cooperator'),
                 "Strat_Defect": lambda m: m.collect_strat_amount('Defector'),
                 "Strat_TFT": lambda m: m.collect_strat_amount('Tit For Tat'),
                 "Strat_Random": lambda m: m.collect_strat_amount('Random'),
                 "Strat_WSLS": lambda m: m.collect_strat_amount('Win-Stay Lose-Shift'),
                 "Cooperators": lambda m: m.get_cooperators(),
                 "Defectors": lambda m: m.get_defectors()}

# Create batch
batch = BatchRunner(MesaAxelrodNetwork, 
                        max_steps=max_steps,
                        variable_parameters={name:[] for name in problem['names']},
                        model_reporters=model_reporters)

# To store data
count = 0
data = pd.DataFrame(index=range(replicates*len(param_values)), 
                                columns=['rationality', 'rank', 'switch_random'])

data['Run'], data['Strat_Cooperate'], data['Strat_Defect'] = None, None, None
data['Strat_TFT'], data['Strat_Random'], data['Strat_WSLS'] = None, None, None
data['Cooperators'], data['Defectors'] = None, None, 

# For set amount
for i in range(replicates):
    
    # For sample
    for vals in param_values: 
        
        # Transform to dict with parameter names and their values
        variable_parameters = {}
        for name, val in zip(problem['names'], vals):
            variable_parameters[name] = val
        
        # Run batch, get data, store data
        batch.run_iteration(variable_parameters, tuple(vals), count)
        iteration_data = batch.get_model_vars_dataframe().iloc[count]
        iteration_data['Run'] = count 
        data.iloc[count, 0:3] = vals
        data.iloc[count, 3:11] = iteration_data
        count += 1

        clear_output()
        print(f'{count / (len(param_values) * (replicates)) * 100:.2f}% done')
        
# Save data to separate file 
data.to_csv("Sobol_all.csv")